# Fundamentos de Organización de Datos
## Propuesta de Cátedra

### Introducción
El principal eje propuesto en este documento trata de presentar un enfoque incremental para la organización de información en memoria secundaria. El principal hilo consta del siguiente flujo

- Serialización: Como transformamos la data en una forma normal y distribuible independiente del lenguaje y de la memoria princiapl
  - JSON
  - XML
  - YAML
  - Struct
  - Pickle
- Archivos estructurados: Dado una fuerte serialización binaria estructurada, se propone implementar un mecanismo de archivos de longitud fija para almacenar la data serializada. Y el mecanismo inverso
- Archivos dinámicos: Dado un buen esquema estructurado se propone un esquema de longitud variable para almacenar la información de forma binaria
- Archivos de Texto: En un enfoque de colavoración se aplican tecnicas de serialización a strings en archivos normalizados entendibles por el usuario
- Aplicaciones en el mundo real
  - API
  - Configuraciones
  - Reportes

### Serialización
La serialización *(serialization)* es el proceso de transformar las estructuras de datos en un formato que puede ser guardado o transmitido y luego reconstruido.

El proceso contrario, de extraer la estructura de datos a partir de una secuencia de bytes, se lo conoce como deserialización *(deserialization/unmarshalling)*

Un serializador puede estar descompuesto en dos pasos:
- Normalización: Transformar la data en una estructura normal de tipo `(clave, valor)`.
- Encodificación: Transformar la estructura normal en una cadena de bytes transmisible.

![Symfony Serializer Component](https://symfony.com/doc/current/_images/serializer_workflow.png)

Algunos ejemplos conocidos de formatos de encodificación, entre otros, son:
- JSON
- XML
- YAML
- Struct
- Pickle
- Ston

Nosotros definiremos las dos operaciones de un serializador de la forma:
- `serialize(data)`: Responsable de transformar un dato a un formato guardable
- `deserialize(buffer)`: Responsable de transformar un formato guardable a una estructura de datos

#### Ejemplos
El primer ejemplo a proponer será serializar una tupla de dos valores enteros `(x, y)` a una estructura binaria compuesta por un entero, un separador en blanco y otro entero `b{ixi}`

In [94]:
from struct import Struct

In [95]:
# definimos la estructura binaria
a = Struct('ixi')

def serialize(*data):
    # enpaquetamos los elementos de la tupla en la estructura
    return a.pack(*data)

def deserialize(buffer):
    # desempaquetamos los elementos del buffer en base a la estructura
    return a.unpack(buffer)

In [96]:
serialize(10, 154)

b'\n\x00\x00\x00\x00\x00\x00\x00\x9a\x00\x00\x00'

In [97]:
deserialize(b'\n\x00\x00\x00\x00\x00\x00\x00\x9a\x00\x00\x00')

(10, 154)

Un ejemplo para la serialización del objeto `Point` a JSON podría ser:

In [98]:
import json

In [99]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class PointSerializer:
    def serialize(self, aPoint):
        return self.encode(self.normalize(aPoint))
    
    def deserialize(self, aJson):
        return self.denormalize(self.decode(aJson))
    
    def encode(self, data):
        return json.dumps(data)
    
    def decode(self, aJson):
        return json.loads(aJson)
    
    def normalize(self, aPoint):
        return { 'x': aPoint.x, 'y': aPoint.y }
    
    def denormalize(self, data):
        return Point(x=data['x'], y=data['y'])

In [100]:
p1 = Point(1, 5)
p2 = Point(-2, 3)

serializer = PointSerializer()
print(serializer.serialize(p1))
print(serializer.serialize(p2))

p3 = serializer.deserialize('{"x": 10, "y": 20}')
print(p3.x)
print(p3.y)

{"x": 1, "y": 5}
{"x": -2, "y": 3}
10
20


### Archivos Estructurados
Como vimos en la sección anterior, logramos obtener una representación binaria de una tupla de dos enteros.

Es logico entonces pensar que podemos estructurar un archivo tal que guarde un conjunto de dichos binarios para luego entenderlos como tuplas.

Llamamos entonces, archivo estructurado a aquel archivo que almacena elementos de una estructura determinada, y cada elemento tiene el mismo tamaño determinante.

Luego podemos decir que el i-ésimo elemento en ese archivo se encuentra en la posición `i*n` y se encuentra en los proximos `n` bytes; siendo `n` el tamaño de la estructura

En python, un archivo se abre bajo la función `open(filename, mode)`. En particualr nuestro archivo contendrá elementos binarios y realizaremos operaciones de escritura y lectura; por lo tanto el modo será `'wb+'`.
- `w`: Para tener privilegios de escritura
- `b`: Por ser binario
- `+`: Para escalar a lectura

In [101]:
file = open('numeros.dat', 'rb+')

In [102]:
file.seek(0)
file.write(serialize(10, 10))
file.write(serialize(-5, 20))
file.write(serialize(-5, 20))
file.seek(0)
while True:
    try:
        # se lee tantos bytes como el tamaño de la estructura
        print(deserialize(file.read(a.size) )) # se deserializa el buffer
    except:
        break # Se sale del while por EOF

(10, 10)
(-5, 20)
(-5, 20)


#### Ejercicios 

In [103]:
with open('e1.dat', 'wb+') as file:
    a = Struct('i')
    file.write(a.pack(10))
    file.write(a.pack(15))
    file.write(a.pack(1))
    file.write(a.pack(5))
    file.write(a.pack(13))
with open('e2.dat', 'wb+') as file:
    a = Struct('i')
    file.write(a.pack(1))
    file.write(a.pack(2))
    file.write(a.pack(3))
    file.write(a.pack(4))
    file.write(a.pack(5))

In [104]:
def ejercicio1():
    '''
    Encontrar el maximo numero en un archivo de enteros
    '''
    a = Struct('i')
    maximo = 0
    file = open('e1.dat', 'rb')
    while True:
        try:
            n, = a.unpack(file.read(a.size))
            maximo = max(n, maximo)
        except:
            break
    print(maximo)
ejercicio1()

15


In [105]:
def ejercicio2():
    '''
    Actualizar un archivo de enteros. Dado cada elemento, su doble
    '''
    a = Struct('i')
    file = open('e2.dat', 'rb+')
    while True:
        try:
            n, = a.unpack(file.read(a.size))
            file.seek(file.tell() - a.size)
            file.write(a.pack(n*2))
        except:
            break
ejercicio2()
with open('e2.dat', 'rb') as file:
    try: 
        while True:
            print(Struct('i').unpack(file.read(4)))
    except:
        pass

(2,)
(4,)
(6,)
(8,)
(10,)
